In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

import capslayer as cl
import tensorflow as tf
import Model_functions as mf
import Dataset
import numpy as np

C:\Program Files\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Data Pipeline

In [2]:
#  Set other variables
sample_rate=16384
n_fft=1024
fft_hop=256
patch_window=256
patch_hop=128
n_parallel_readers=4
normalise=True
batch_size = 5
shuffle=False
n_shuffle = 10

directory_a = 'C:/Users/Toby/MSc_Project/Test_Audio/GANdatasetsMini/test/Mixed'
directory_b = 'C:/Users/Toby/MSc_Project/Test_Audio/GANdatasetsMini/test/Voice'

#  Create the pipeline
tf.reset_default_graph()
data = Dataset.zip_files(directory_a, directory_b)
data = Dataset.get_paired_dataset(data,
                                  sample_rate,
                                  n_fft,
                                  fft_hop,
                                  patch_window,
                                  patch_hop,
                                  n_parallel_readers,
                                  batch_size,
                                  n_shuffle,
                                  normalise)

#  Create the iterator
mixed_spec, voice_spec, mixed_audio, voice_audio = data.make_one_shot_iterator().get_next()

#  Create variable placeholders
is_training = tf.placeholder(shape=(), dtype=bool)
mixed_mag = tf.expand_dims(mixed_spec[:, :, 1:, 0], 3)
mixed_phase = tf.expand_dims(mixed_spec[:, :, 1:, 1], 3)
voice_mag = tf.expand_dims(voice_spec[:, :, 1:, 0], 3)

## CapsNet Definition

In [3]:
class basicCapsNet(object):
    
    def __init__(self, mixed_mag, voice_mag, is_training, reuse=True, name='basic_caps_net'):
        """
        input_tensor: Tensor with shape [batch_size, height, width, channels]
        is_training:  Boolean - should the model be trained on the current input or not
        name:         Model instance name
        """
        with tf.variable_scope(name):
            self.mixed_mag = mixed_mag
            self.voice_mag = voice_mag
            self.is_training = is_training
            
            with tf.variable_scope('Convolution'):
                net = mf.conv(mixed_mag, filters=128, kernel_size=5, stride=(1, 1))
                self.conv1 = net
                
            with tf.variable_scope('Primary_Caps'):
                # poses is the output of each capsule in the layer. Represents the state of the feature which the capsule detects
                # probs is the detection probability for each capsule, corresponding to the magnitude of poses
                poses, probs = cl.layers.primaryCaps(inputs=net, 
                                                     filters=16, 
                                                     kernel_size=5, 
                                                     strides=(1,1),
                                                     out_caps_dims=[8,1], 
                                                     method='norm')
                self.primary_caps = (poses, probs)
            
            with tf.variable_scope('Conv_Caps'):
                poses, probs = cl.layers.conv2d(inputs=poses,
                                                activation=probs,
                                                filters=1,
                                                out_caps_dims=[16,1],
                                                kernel_size=1,
                                                strides=(1,1),
                                                padding="valid",
                                                routing_method="DynamicRouting",
                                                reuse=None)
                self.conv_caps = (poses, probs)
            """"""
            #with tf.variable_scope('Reconstruction_Conv'): # Output should be (?, 256, 512, 128)
            #    net = 
                
            #with tf.vriable_scope('Mask_Constructor'): # Output.shape == mixed_mag.shape ((?, 256, 512, 1))
            # Output of caps layers needs to be?
            #           Mask of shape [input_tensor]
            self.voice_mask = net
            #self.gen_voice = self.voice_mask * mixed_mag
            
            #self.cost = ####
            #self.optimizer = ####
            #self.training_op = ####

In [4]:

model = basicCapsNet(mixed_mag, voice_mag, is_training=False)

In [5]:
model.conv_caps

(<tf.Tensor 'basic_caps_net/Conv_Caps/conv2d/routing/Squeeze:0' shape=(?, 252, 508, 1, 16, 1) dtype=float32>,
 <tf.Tensor 'basic_caps_net/Conv_Caps/conv2d/clip_by_value:0' shape=(?, 252, 508, 1) dtype=float32>)

Network so far:
- Input: (?, 256, 512, 1)
- Convolution: (?, 252, 508, 128)
- Primary_Caps: (?, 252, 508, 16, 8, 1), (?, 252, 508, 16)
- Conv_Caps: (?, 252, 508, 16, 1), (?, 252, 508, 1)

Up-ConvCaps layer needs to:
- take - 
    - input: [batch size, height, width, channels (capsule layer count), caps dims 1, caps dim 2]
    - activtion: [batch size, height, width, channels (capsule layer count)]
    - output dims: [channels (capsule layer count), caps dims 1, caps dim 2]
    - strides
    - padding
- return - 
    - output: [batch size, height, width, channels (capsule layer count), caps dims 1, caps dim 2]

Conv_caps to Conv layer needs to:
- take - 
    - input: 6D Tensor, [batch size, height, width, channels (capsule layer count), caps dims 1, caps dim 2]
    - activtion: 4D Tensor, [batch size, height, width, channels (capsule layer count)]
    - output filters: int, channels (convolutional filter count)
    - strides
    - padding
- return - 
    - output: 4D Tensor, [batch size, height, width, channels (convolutional filter count)]

Mask reconstruction layer needs to:
- take - 
    - input: 4D Tensor, [batch size, height, width, channels (convolutional filter count)]
    - activtion: [batch size, height, width, channels (capsule layer count)]
    - output filters: int, channels (depth of original input, generally one, but flexibility would be good)
    - strides
    - padding
- return - 
    - output: 4D Tensor, [batch size, height, width, channels (depth of original input, generally one, but flexibility would be good)]

Can this be a normal tf.conv2d_transpose?
- Args -
    - value: 4D Tensor, [batch, height, width, in_channels]
    - filter: 4D Tensor, [height, width, output_channels, in_channels]
    - output_shape: 1-D Tensor representing the output shape of the deconvolution op
    - strides
    - padding
- Returns -
    - A Tensor with the same type as value

In [ ]:
mixed_mag

In [ ]:
test = tf.reshape(mixed_mag, shape=[-1, mixed_mag.shape[1], mixed_mag.shape[2], mixed_mag.shape[3]])

In [ ]:
test

In [ ]:
model.primary_caps[1]

In [ ]:
test = transforming(model.voice_mask,1, [16,1])

In [ ]:
test

In [ ]:
test2 = cl.layers.conv2d(inputs=model.primary_caps[0],
                         activation=model.primary_caps[1],
                         filters=1,
                         out_caps_dims=[16,1],
                         kernel_size=1,
                         strides=(1,1),
                         padding="valid",
                         routing_method="DynamicRouting",
                         name=None,
                         reuse=None)
test2

In [ ]:
spatial_shape = [1, 2, 3, 4]
input_shape = [5,5,5]

In [ ]:
[-1] + spatial_shape + input_shape[1:]

In [6]:
def space_to_batch_nd_v1(inputs, kernel_size, strides, name=None):
    """ for convCapsNet model: memory 4719M, speed 0.169 sec/step
    """
    name = "space_to_batch_nd" if name is None else name
    with tf.name_scope(name):
        height, width, depth = cl.shape(inputs)[1:4]
        h_offsets = [[(h + k) for k in range(0, kernel_size[0])] for h in range(0, height + 1 - kernel_size[0], strides[0])]
        w_offsets = [[(w + k) for k in range(0, kernel_size[1])] for w in range(0, width + 1 - kernel_size[1], strides[1])]
        d_offsets = [[(d + k) for k in range(0, kernel_size[2])] for d in range(0, depth + 1 - kernel_size[2], strides[2])]
        patched = tf.gather(inputs, h_offsets, axis=1)
        patched = tf.gather(patched, w_offsets, axis=3)
        patched = tf.gather(patched, d_offsets, axis=5)

        if len(patched.shape) == 7:
            perm = [0, 1, 3, 5, 2, 4, 6]
        else:
            perm = [0, 1, 3, 5, 2, 4, 6, 7, 8]

        patched = tf.transpose(patched, perm=perm)
        shape = cl.shape(patched)

        if depth == kernel_size[2]:   # for conv2d
            shape = shape[:3] + [np.prod(shape[3:-2])] + shape[-2:] if len(patched.shape) == 9 else shape[:3] + [np.prod(shape[3:])]
        else:                         # for conv3d
            shape = shape[:4] + [np.prod(shape[4:-2])] + shape[-2:] if len(patched.shape) == 9 else shape[:4] + [np.prod(shape[4:])]

        patched = tf.reshape(patched, shape=shape)
    return patched

In [8]:
def batch_to_space_nd(input, spatial_shape, name=None):
    name = "batch_to_space_nd" if name is None else name
    with tf.name_scope(name):
        input_shape = cl.shape(input)
        shape = [-1] + spatial_shape + input_shape[1:]
        return tf.reshape(input, shape=shape)

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
inputs = model.primary_caps[0]
kernel_size = [5,5,1]
strides = [2,2,1]

In [10]:
patched_1 = space_to_batch_nd_v1(inputs, kernel_size, strides)
print(patched_1)

Tensor("space_to_batch_nd/Reshape:0", shape=(?, 124, 252, 16, 25, 8, 1), dtype=float32)


In [ ]:
inputs

In [ ]:
reconstruction = batch_to_space_nd(batched, kernel_size)
reconstruction

In [ ]:
input_shape = cl.shape(batched)
input_shape[1:]

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
inputs = model.primary_caps[0]
height, width, depth = cl.shape(inputs)[1:4]
kernel_size = [5,5,1]
strides = [2,2,1]

In [ ]:
h_offsets = [[(h + k) for k in range(0, kernel_size[0])] for h in range(0, height + 1 - kernel_size[0], strides[0])]
w_offsets = [[(w + k) for k in range(0, kernel_size[1])] for w in range(0, width + 1 - kernel_size[1], strides[1])]
d_offsets = [[(d + k) for k in range(0, kernel_size[2])] for d in range(0, depth + 1 - kernel_size[2], strides[2])]

In [ ]:
patched = tf.gather(inputs, h_offsets, axis=1)
patched

In [ ]:
patched = tf.gather(patched, w_offsets, axis=3)
patched

In [ ]:
patched = tf.gather(patched, d_offsets, axis=5)
patched

In [ ]:
if len(patched.shape) == 7:
    perm = [0, 1, 3, 5, 2, 4, 6]
else:
    perm = [0, 1, 3, 5, 2, 4, 6, 7, 8]
patched = tf.transpose(patched, perm=perm)
shape = cl.shape(patched)
shape

In [ ]:
shape[:3] + [np.prod(shape[3:])]

In [ ]:
import numpy as np
shape = shape[:3] + [np.prod(shape[3:-2])] + shape[-2:] if len(patched.shape) == 9 else shape[:3] + [np.prod(shape[3:])]
shape

In [ ]:
patched = tf.reshape(patched, shape=shape)
patched

In [ ]:
h_offsets

In [ ]:
import capslayer as cl
import tensorflow as tf

from capslayer.core import routing
from capslayer.core import transforming


def conv2d_transpose(inputs,
                     activation,
                     filters,
                     out_caps_dims,
                     kernel_size,
                     strides,
                     padding="valid",
                     routing_method="EMRouting",
                     name=None,
                     reuse=None):
    """A 2D transposed convolutional (sometimes called deconvolutional) capsule layer.
    Args:
        inputs: A 6-D tensor with shape [batch_size, in_height, in_width, in_channels] + in_caps_dims.
        activation: A 4-D tensor with shape [batch_size, in_height, in_width, in_channels].
        filters: Integer, the dimensionality of the output space (i.e. the number of filters in the convolution).
        out_caps_dims: A tuple/list of 2 integers, specifying the dimensions of output capsule, e.g. out_caps_dims=[4, 4] representing that each output capsule has shape [4, 4].
        kernel_size:  An integer or tuple/list of 2 integers, specifying the height and width of the 2D convolution window. Can be a single integer to specify the same value for all spatial dimensions.
        strides: An integer or tuple/list of 2 integers, specifying the strides of the convolution along the height and width. Can be a single integer to specify the same value for all spatial dimensions.
        padding: One of "valid" or "same" (case-insensitive), now only support "valid".
        routing_method: One of "EMRouting" or "DynamicRouting", the method of routing-by-agreement algorithm.
        name: A string, the name of the layer.
        reuse: Boolean, whether to reuse the weights of a previous layer by the same name.
    Returns:
        pose: A 6-D tensor with shape [batch_size, out_height, out_width, out_channesl] + out_caps_dims.
        activation: A 4-D tensor with shape [batch_size, out_height, out_width, out_channels].
    """

    name = "conv2d" if name is None else name
    with tf.variable_scope(name) as scope:
        if reuse:
            scope.reuse_variables()

        input_shape = cl.shape(inputs)
        input_rank = len(input_shape)
        activation_rank = len(activation.shape)
        if not input_rank == 6:
            raise ValueError('Inputs to `conv2d` should have rank 6. Received inputs rank:', str(input_rank))
        if not activation_rank == 4:
            raise ValueError('Activation to `conv2d` should have rank 4. Received activation rank:', str(activation_rank))

        if isinstance(kernel_size, int):
            kernel_size = [kernel_size, kernel_size, input_shape[3]]
        elif isinstance(kernel_size, (list, tuple)) and len(kernel_size) == 2:
            kernel_size = [kernel_size[0], kernel_size[1], input_shape[3]]
        else:
            raise ValueError('"kernel_size" should be an integer or tuple/list of 2 integers. Received:', str(kernel_size))

        if isinstance(strides, int):
            strides = [strides, strides, 1]
        elif isinstance(strides, (list, tuple)) and len(strides) == 2:
            strides = [strides[0], strides[1], 1]
        else:
            raise ValueError('"strides" should be an integer or tuple/list of 2 integers. Received:', str(kernel_size))

        if not isinstance(out_caps_dims, (list, tuple)) or len(out_caps_dims) != 2:
            raise ValueError('"out_caps_dims" should be a tuple/list of 2 integers. Received:', str(out_caps_dims))
        elif isinstance(out_caps_dims, tuple):
            out_caps_dims = list(out_caps_dims)

        # 1. space to batch
        # patching everything into [batch_size, out_height, out_width, in_channels] + in_caps_dims (batched)
        # and [batch_size, out_height, out_width, in_channels] (activation).
        batched = cl.space_to_batch_nd(inputs, kernel_size, strides)
        activation = cl.space_to_batch_nd(activation, kernel_size, strides)

        # 2. transforming
        # transforming to [batch_size, out_height, out_width, in_channels, out_channels/filters] + out_caps_dims
        vote = transforming(batched,
                            num_outputs=filters,
                            out_caps_dims=out_caps_dims)

        # 3. routing
        pose, activation = routing(vote, activation, method=routing_method)

        return pose, activation